In [1]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import files
uploaded=files.upload()

Saving zerotestbiglist.json to zerotestbiglist.json
Saving zerotrainbiglist.json to zerotrainbiglist.json


In [4]:
def load_json(input_file):
    with open(input_file) as f:
    
        reading = json.load(f)
    return reading

train_list = np.array(load_json("zerotrainbiglist.json"))
test_list = np.array(load_json("zerotestbiglist.json"))

train_X = np.expand_dims(train_list,axis=-1)
test_X = np.expand_dims(test_list,axis=-1)

In [5]:
#build the encoder
inputs = Input((28,28,1))
x = inputs

x = Conv2D(32, (3,3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2D(64, (3,3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

volumeSize = K.int_shape(x)
x = Flatten()(x)
latent = Dense(160)(x)
encoder = Model(inputs, latent, name="encoder")

print(encoder.summary())

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 64)          256 

In [7]:
#build the decoder
latentInputs = Input(shape=(160,))
x = Dense(np.prod(volumeSize[1:]))(latentInputs)
x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

x = Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2DTranspose(1, (3, 3), padding="same")(x)
outputs = Activation("sigmoid")(x)

decoder = Model(latentInputs, outputs, name="decoder")
print(decoder.summary())

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 160)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 3136)              504896    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 28, 28, 32)        1846

In [8]:
#build the autoencoder
mautoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
print(mautoencoder.summary())

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Functional)         (None, 160)               521120    
_________________________________________________________________
decoder (Functional)         (None, 28, 28, 1)         560961    
Total params: 1,082,081
Trainable params: 1,081,697
Non-trainable params: 384
_________________________________________________________________
None


In [9]:
mautoencoder.compile(optimizer=Adam(lr=1e-3), loss="mean_squared_error")

In [11]:
(mX_train, mY_train), (mX_test, mY_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [12]:
mX_train = mX_train.reshape(60000,28,28,1)
mX_test = mX_test.reshape(10000,28,28,1)

In [13]:
print(mX_train.shape)
print(mX_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [15]:
cv2_imshow(mX_test[120]*255)

In [ ]:
#train the model
history = mautoencoder.fit(
	mX_train, mX_train,
	validation_data=(mX_test, mX_test),
	epochs=25)

Epoch 1/25
1875/1875 [==============================] - 41s 4ms/step - loss: 7226.7017 - val_loss: 7342.8442
Epoch 2/25
1875/1875 [==============================] - 7s 4ms/step - loss: 7201.9652 - val_loss: 7342.8145
Epoch 3/25
1875/1875 [==============================] - 8s 4ms/step - loss: 7236.3113 - val_loss: 7342.8071
Epoch 4/25
1875/1875 [==============================] - 8s 4ms/step - loss: 7212.9454 - val_loss: 7342.7969
Epoch 5/25
1875/1875 [==============================] - 8s 4ms/step - loss: 7208.7568 - val_loss: 7342.7959
Epoch 6/25
1875/1875 [==============================] - 8s 4ms/step - loss: 7205.6736 - val_loss: 7342.7930
Epoch 7/25
1875/1875 [==============================] - 8s 4ms/step - loss: 7224.7482 - val_loss: 7342.7808
Epoch 8/25
1875/1875 [==============================] - 8s 4ms/step - loss: 7230.5095 - val_loss: 7342.7842
Epoch 9/25
1875/1875 [==============================] - 7s 4ms/step - loss: 7214.2328 - val_loss: 7342.7778
Epoch 10/25
1875/1875 [====

In [32]:
cv2_imshow(mX_test[7]*255)

In [33]:
test_mnist = np.reshape(mX_test[7], (1,28,28,1))
output_mnist = mautoencoder.predict(test_mnist)
predicted_mnist = np.reshape(output_mnist,(28,28,1))*255
cv2_imshow(predicted_mnist)

In [20]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)             

In [ ]:
model.add()

In [22]:
(mX_train, mY_train), (mX_test, mY_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [23]:
print(mX_train.shape)
print(mX_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [24]:
mX_train = mX_train.reshape(60000,28,28,1)
mX_test = mX_test.reshape(10000,28,28,1)

In [25]:
print(mX_train.shape)
print(mX_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [ ]:
#train the model
history = autoencoder.fit(
	train_X, train_X,
	validation_data=(test_X, test_X),
	epochs=20)

Epoch 1/20


ResourceExhaustedError: ignored

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
xc=range(100)

In [ ]:
#plot the loss
plt.figure(1, figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'])
plt.style.use(['classic'])

In [ ]:
#plot the accuracy
plt.figure(2, figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'],loc=3)
plt.style.use(['classic'])

In [ ]:
autoencoder.save('trained_autoencoder.h5')

In [ ]:
def draw_grid(out):
  res=out.shape
  plt.imshow(out, cmap = "PiYG_r")
  plt.clim(0.0, 1.0)
  plt.gca().set_xticks(np.arange(-.5, res[1], 2), minor = True)
  plt.gca().set_yticks(np.arange(-.5, res[0], 2), minor = True)
  plt.grid(True, which="minor", color="w", linewidth = .6, alpha = 0.5)
  plt.colorbar()
  plt.show()

In [ ]:
decoded = autoencoder.predict(testX)